In [70]:
import plotly.graph_objects as go
import pandas as pd

import plotly.express as px

import plotly.offline as offline

from ipywidgets import widgets

from datetime import datetime


In [181]:
data= pd.read_csv("assets/bond_details.csv", sep=",", parse_dates=["Date_of_Purchase"])
data["Denomination"] = data["Denomination"].astype(float)

In [182]:

data['Month_Year'] = data['Date_of_Purchase'].dt.strftime('%b-%Y')
#data['Month_Year'] = data['Date_of_Purchase'].dt.to_period('m')
#data['Month_Year'] = pd.to_datetime(data['Month_Year'], format='%b-%Y')

In [155]:
## Normal overall analysis of the bonds.
# We group data by purchasers name.

bonds_tree= data.groupby("Purchaser_Name")["Denomination"].apply(sum).to_dict()  # only use it for a combined analysis of all bonds
# We can use the following code to create a treemap of the bonds.         
fig=go.Figure(go.Treemap(labels= list(bonds_tree.keys()), parents= [""]*len(bonds_tree.keys()), values= list(bonds_tree.values()), text= list(bonds_tree.keys()), marker_colors=px.colors.qualitative.Plotly, textinfo="label+text+value"))    
fig.update_layout(width=750, height=750, margin=dict(l=0, r=0, t=0, b=0))
    
offline.plot(fig, filename="ipy_bonds_map.html", auto_open=True)

'ipy_bonds_map.html'

In [156]:
# Sample DataFrame
#datetime.datetime.strptime(year.value[0], "%b-%Y")

# Function to update the treemap
def update_treemap(y):
    year = selection_range_slider.value

    idx_start = data.index[data['Month_Year'] == year[0]][0]
    idx_end = data.index[data['Month_Year'] == year[1]][-1]

    filtered_df = data.loc[idx_start : idx_end]
    #filtered_df= data.between_time(parser.isoparse(year.value[0]), parser.isoparse(year.value[1]))
    #filtered_df = data.loc[data['Month_Year'] >= datetime.strptime(year.value[0], "%b-%Y") & data['Month_Year'] <= datetime.strptime(year.value[1], "%b-%Y")]
    #filtered_df = data.loc[(datetime.strptime(data['Month_Year'],"%b-%Y") >= datetime.strptime(year.value[0], "%b-%Y")) & (datetime.strptime(data['Month_Year'],"%b-%Y") <= datetime.strptime(year.value[1], "%b-%Y"))]
    dict_b= filtered_df.groupby("Purchaser_Name")["Denomination"].apply(sum).to_dict()
    fig = go.Figure(go.Treemap(
        labels = list(dict_b.keys()),
        parents = [""]*len(dict_b.keys()),   #[""]*len(bonds_tree.keys()
        values = list(dict_b.values()),
        textinfo = "label+value"
    ))
    fig.update_layout( width=1000 , height= 1000 , margin = dict(t=25, l=25, r=25, b=25))
    return fig

# Slider widget for year selection
selection_range_slider = widgets.SelectionRangeSlider(
    options=unique_dates,
    index=(0, len(unique_dates)-1),
    start=0,
    end=len(unique_dates)-1,
    #value=('Apr-2019', 'Oct-2021'),
    step=1,
    description='Dates',
    orientation='horizontal',
    layout={'width': '1000px'},
    continuous_update=False,
)

#widgets.interactive(update_treemap, y=selection_range_slider)

widgets.interact(update_treemap, y=selection_range_slider)
#widgets.widget_layout.Layout(display='flex', justify_content='space-between', align_items='center', width='30%')
#embed_minimal_html('export.html', views=[update_treemap, selection_range_slider], title='Widgets export')

interactive(children=(SelectionRangeSlider(continuous_update=False, description='Dates', index=(0, 21), layout…

<function __main__.update_treemap(y)>

In [106]:
data2= pd.read_csv("assets/party_details.csv", sep=",", parse_dates=["Date_of_Encashment"])
data2["Denomination"] = data2["Denomination"].astype(float)

In [108]:
data2['Month_Year'] = data2['Date_of_Encashment'].dt.strftime('%b-%Y')

In [158]:
data.Denomination.unique()

array([1.e+05, 1.e+06, 1.e+07, 1.e+03, 1.e+04])

In [174]:

date_dict= data.groupby("Month_Year")["Denomination"].size().to_dict()
date_dict

{'Apr-2019': 1335,
 'Apr-2021': 974,
 'Apr-2022': 811,
 'Apr-2023': 1470,
 'Dec-2022': 260,
 'Jan-2020': 139,
 'Jan-2021': 151,
 'Jan-2022': 2068,
 'Jan-2023': 437,
 'Jan-2024': 897,
 'Jul-2019': 101,
 'Jul-2021': 465,
 'Jul-2022': 480,
 'Jul-2023': 1371,
 'May-2019': 1187,
 'Nov-2022': 954,
 'Nov-2023': 1109,
 'Oct-2019': 531,
 'Oct-2020': 321,
 'Oct-2021': 1057,
 'Oct-2022': 741,
 'Oct-2023': 2012}

In [175]:
#filtered_df = data.loc[data['Month_Year'] == 'Apr-2019']
df_grouped = data.groupby(['Month_Year', 'Denomination']).size().unstack(fill_value=0)

In [183]:
data['dt_month_year'] = data['Date_of_Purchase'].dt.to_period('M')

In [177]:

# Normalize counts to percentages
df_grouped_percent = df_grouped.div(df_grouped.sum(axis=1), axis=0) * 100

# Create stacked area chart
trace1 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[1000],
    mode='lines',
    stackgroup='one',
    name='1 Thousand'
)
trace2 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[10000],
    mode='lines',
    stackgroup='one',
    name='10 Thousand'
)
trace3 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[100000],
    mode='lines',
    stackgroup='one',
    name='1 Lakh'
)
trace4 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[1000000],
    mode='lines',
    stackgroup='one',
    name='10 Lakh'
)
trace5 = go.Scatter(
    x=df_grouped_percent.index,
    y=df_grouped_percent[10000000],
    mode='lines',
    stackgroup='one',
    name='1 Crore'
)

data = [trace1, trace2, trace3, trace4, trace5]

layout = go.Layout(
    title='Type of Denominations (Percentage)',
    xaxis=dict(title='Month Year'),
    yaxis=dict(title='Percentage'),
    hovermode='x'
)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [185]:
data2

,Date_of_Encashment,Name_of_Party,Denomination,Month_Year
0,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0,Apr-2019
1,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0,Apr-2019
2,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,10000000.0,Apr-2019
3,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0,Apr-2019
4,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.0,Apr-2019
...,...,...,...,...
20416,2024-01-24,JANASENA PARTY,1000000.0,Jan-2024
20417,2024-01-24,JANASENA PARTY,1000000.0,Jan-2024
20418,2024-01-24,JANASENA PARTY,1000000.0,Jan-2024
20419,2024-01-24,JANASENA PARTY,1000000.0,Jan-2024
